# Cvičení 2: Drift trosek na hladině

**Hlavní myšlenka:** Lehké těleso na hladině je unášeno rychlostí prostředí (vítr, proud). Budeme modelovat jeho pohyb v rovině a postavíme jednoduchou numerickou simulaci.

**Cíle:**
- umět formálně popsat polohu $\mathbf{r}(t)=(x(t),y(t))$, rychlost $\mathbf{v}(t)$ a zrychlení $\mathbf{a}(t)$ ve 2D,
- vysvětlit vztah mezi zrychlením $\mathbf{a}$ a odporovou silou působící na těleso,
- chápat rychlost prostředí jako vektorové pole $\mathbf{w}(\mathbf{r}, t)$ [m/s],
- převést relativní rychlost $\mathbf{v}-\mathbf{w}$ na odporovou sílu,
- zjednodušit popis vlastností unášené trosky na jediný parametr $\tau$ (časová konstanta odporu),
- použít Eulerovu metodu pro rychlost a lichoběžníkové pravidlo pro polohu.

**Zadání:** Částice se od startu $(x_0, y_0)$ v čase $t_0$ s nulovou počáteční rychlostí $\mathbf{v}_0=(0,0)$ pohybuje v prostředí s rychlostí $\mathbf{w}(\mathbf{r}, t)$. Jediná síla je odpor úměrný relativní rychlosti. Výstupem je trajektorie do času $t_1$.

**Poznámka:** Řešení budeme skládat postupně v několika úkolech.


---


## Okamžitá poloha a okamžitá rychlost

**Hlavní myšlenka:** Stav částice popisujeme vektorem polohy v rovině
$$
\mathbf{r}(t) = (x(t), y(t)).
$$
Okamžitá rychlost a zrychlení jsou derivace podle času:
$$
\mathbf{v}(t) = \mathbf{r}'(t), \qquad \mathbf{a}(t) = \mathbf{v}'(t).
$$
Rychlost určuje směr a velikost pohybu, zrychlení popisuje, jak se rychlost mění v čase.

**Doplňující kontext:** Obrázky níže ukazují jednoduchý 2D pohyb s konstantním zrychlením; všechny grafy odpovídají jednomu a témuž pohybu.

<img src="images/kinematika_trajektorie.png" alt="Trajektorie v rovině x–y" width="480"/>

<table>
  <tr>
    <td><img src="images/kinematika_x_t.png" alt="x(t)" width="320"/></td>
    <td><img src="images/kinematika_y_t.png" alt="y(t)" width="320"/></td>
  </tr>
  <tr>
    <td><img src="images/kinematika_vx_t.png" alt="v_x(t)" width="320"/></td>
    <td><img src="images/kinematika_vy_t.png" alt="v_y(t)" width="320"/></td>
  </tr>
  <tr>
    <td><img src="images/kinematika_ax_t.png" alt="a_x(t)" width="320"/></td>
    <td><img src="images/kinematika_ay_t.png" alt="a_y(t)" width="320"/></td>
  </tr>
</table>


---


## Numerický výpočet trajektorie ze znalosti rychlosti

**Hlavní myšlenka:** Výpočet polohy děláme po složkách. Stačí odvodit vztah v 1D (např. pro x); pro y je postup stejný. Obrázek níže je 1D ukázka principu.

Základní vztah pro polohu je
$$
\mathbf{r}(t)=\mathbf{r}_0+\int_{t_0}^{t}\mathbf{v}(\tau)\,d\tau.
$$

**Diskrétní čas (po složkách):** Zavedeme časové body
$$
t_i=t_0+i\,\Delta t, \qquad i=0,1,\dots,N.
$$
Rychlost známe v těchto bodech: $v_{x,i}=v_x(t_i)$. Cílem je spočítat polohy $x_i=x(t_i)$ ve stejných bodech, přičemž $x_0$ je počáteční poloha.

**Lichoběžníkové pravidlo (pro složku x):**
$$
x_i=x_{i-1}+\frac{v_{x,i-1}+v_{x,i}}{2}\,\Delta t.
$$
Pro složku y platí stejný vztah s $x \to y$ a $v_x \to v_y$.

![Lichoběžníkové pravidlo](images/lichobeznikove_pravidlo.png)


---


In [ ]:
import numpy as np
import matplotlib.pyplot as plt


### Pomocná funkce `vykresli_drift(t, r, v=None, title=None)`

**K čemu je:** Jednotné vykreslení výsledků simulace. První obrázek je trajektorie v rovině (x–y) se startem a koncem. Pokud je zadáno `v`, druhý obrázek je mřížka 2×2 s průběhy x(t), y(t), v_x(t), v_y(t). Bez `v` se vykreslí jen x(t), y(t).

**Vstupy:**
- `t`: 1D pole časů [s]
- `r`: pole tvaru (n, 2) s polohou [m]
- `v`: pole tvaru (n, 2) s rychlostí [m/s] (volitelné)
- `title`: nadpis grafu trajektorie (volitelné)

**Výstup:** Funkce nic nevrací, jen zobrazí grafy.

**Poznámka:** Osy x–y mají stejný poměr, aby se trajektorie nedeformovala.

**Zdroj:** `cv2_utils.py`.


In [ ]:
from cv2_utils import vykresli_drift


## Úkol 1: Trajektorie z dané rychlosti

**Hlavní myšlenka:** Když známe $\mathbf{v}(t)$, stačí ji numericky integrovat pomocí lichoběžníku.

**Zadání:** Použijte konstantní rychlost $\mathbf{v} = (1, 0.3)$ v intervalu $t \in [0, 10]$ s krokem $\Delta t$ a spočtěte polohu. Použijte prealokaci polí (přehlednější a rychlejší).


In [ ]:
def poloha_2d(r0, v, dt):
    """
    Numericky integruje polohu z rychlosti lichoběžníkovým pravidlem.

    Parametry:
        r0 : počáteční poloha [m], pole tvaru (2,)
        v  : rychlost v čase, pole tvaru (n, 2) [m/s]
        dt : krok času [s]

    Návrat:
        r : poloha v čase, pole tvaru (n, 2) [m]
    """
    v = np.asarray(v, dtype=float)
    n = v.shape[0]
    r = np.zeros((n, 2), dtype=float)
    r[0] = np.asarray(r0, dtype=float)

    ###### doplňte kód zde ######
    for i in range(1, n):
        r[i] = r[i - 1] + 0.5 * (v[i - 1] + v[i]) * dt
    #############################

    return r


# Zadání pro test
r0 = np.array([0.0, 0.0])
t0, t1, dt = 0.0, 10.0, 0.1
t = np.arange(t0, t1 + dt, dt)

v_const = np.tile(np.array([1.0, 0.3]), (t.size, 1))
r = poloha_2d(r0, v_const, dt)

vykresli_drift(t, r, v_const, title="Úkol 1: trajektorie z konstantní rychlosti")


---


## Síla jako motivátor pohybu

**Hlavní myšlenka:** Na částici působí pouze lineární odpor vůči prostředí, takže zrychlení je úměrné rozdílu mezi rychlostí prostředí a rychlostí částice. Zavedeme časovou konstantu
$$
\tau = \frac{m}{c},
$$
která v sobě shrnuje vliv hmotnosti a odporu. V simulacích pak používáme už jen $\tau$.

Model zrychlení zapíšeme přímo:
$$
\mathbf{a}(t) = \frac{1}{\tau}\,\big(\mathbf{w}(\mathbf{r}, t) - \mathbf{v}(t)\big).
$$

**Co znamená $\tau$:** časová konstanta [s] – jak rychle se rychlost částice přizpůsobí lokální rychlosti prostředí. Menší $\tau$ znamená rychlejší přizpůsobení, větší $\tau$ pomalejší.

**Poznámka navíc:** Jednotky: $\mathbf{w}$ a $\mathbf{v}$ [m/s], $\mathbf{r}$ [m], $\tau$ [s].

Abychom z modelu zrychlení získali rychlost $\mathbf{v}(t)$, potřebujeme numericky integrovat zrychlení. To uděláme v následující části Eulerovou metodou.


### Odpor prostředí

Lineární odpor vždy působí proti relativní rychlosti $\mathbf{v} - \mathbf{w}$. Pokud je $\mathbf{w}$ konstantní, rychlost se k ní postupně přibližuje:
$$
\mathbf{v}(t) \to \mathbf{w} \quad \text{pro } t\to\infty.
$$
Na grafu velikosti rychlosti uvidíte, že se k $|\mathbf{w}|$ blížíme hladce bez oscilací (při rozumném kroku $\Delta t$).


---


## Numerický výpočet rychlosti ze zrychlení – Eulerova metoda

**Hlavní myšlenka:** Zrychlení známe jako funkci aktuálního stavu, proto rychlost počítáme krok za krokem.

Explicitní Eulerova metoda:
$$
\mathbf{v}_{i} = \mathbf{v}_{i-1} + \mathbf{a}(\mathbf{v}_{i-1}, \mathbf{r}_{i-1}, t_{i-1})\,\Delta t.
$$

**Poznámka navíc:** Euler je jednoduchý, ale citlivý na velikost $\Delta t$. Příliš velký krok vede k velkým chybám i nestabilitě.


---


## Úkol 2: Rychlost ze zrychlení

Použijeme nejjednodušší případ odporu vůči konstantnímu $\mathbf{w}$, tedy $\mathbf{w} = \mathrm{konst}$. Zrychlení není konstantní, protože závisí na rychlosti $\mathbf{v}$, ale Eulerovu metodu můžeme použít přímo.

**Zadání:** Napište Eulerův krok pro rychlost a ověřte jej na konstantním $\mathbf{w}$. Trajektorii pak dopočítejte pomocí funkce z Úkolu 1. Poznámka: $\mathbf{r}$ zde slouží jen kvůli podpisu `a_funkce`; skutečnou trajektorii počítáme zvlášť.


In [ ]:
def euler_rychlost_2d(v0, a_funkce, t, dt):
    """
    Explicitní Euler pro rychlost v 2D.

    Parametry:
        v0       : počáteční rychlost [m/s], pole tvaru (2,)
        a_funkce : funkce a(v, r, t) -> (ax, ay)
        t        : 1D pole časů [s]
        dt       : krok času [s]

    Návrat:
        a, v : zrychlení a rychlost v čase
    """
    t = np.asarray(t, dtype=float)
    n = t.size
    v = np.zeros((n, 2), dtype=float)
    a = np.zeros((n, 2), dtype=float)
    r = np.zeros((n, 2), dtype=float)
    v[0] = np.asarray(v0, dtype=float)

    ###### doplňte kód zde ######
    a[0] = a_funkce(v[0], r[0], t[0])
    for i in range(1, n):
        v[i] = v[i - 1] + a[i - 1] * dt
        r[i] = r[i - 1] + 0.5 * (v[i - 1] + v[i]) * dt
        a[i] = a_funkce(v[i], r[i], t[i])
    #############################

    return a, v


# Jednoduchý test: odpor vůči konstantnímu $w$ (dané $\tau$)
tau_test = 1.6667  # s
w_test = np.array([1.5, 0.3])  # m/s
v0 = np.array([0.0, 0.0])

t0, t1, dt = 0.0, 8.0, 0.05
t = np.arange(t0, t1 + dt, dt)


def a_const(v, r, t):
    return (1.0 / tau_test) * (w_test - v)


a_euler, v_euler = euler_rychlost_2d(v0, a_const, t, dt)


In [ ]:
# Trajektorie z vypočtené rychlosti
r0 = np.array([0.0, 0.0])
r_euler = poloha_2d(r0, v_euler, dt)

vykresli_drift(t, r_euler, v_euler, title="Úkol 2: Euler pro rychlost")


---


## Úkol 3: Drift s konstantní rychlostí prostředí a odporem

**Hlavní myšlenka:** Spojíme Euler pro rychlost a lichoběžníkové pravidlo pro polohu do jedné simulace. V této úloze už skutečně potřebujeme $\mathbf{r}$, protože rychlost prostředí může být funkcí polohy.

**Zadání:** použijte parametry a jednotky uvedené v kódu a sledujte, jak se rychlost přibližuje k $\mathbf{w}$.


In [ ]:
# Parametry systému
x0, y0 = 0.0, 0.0
r0 = np.array([x0, y0])

v0 = np.array([0.0, 0.0])
t0 = 0.0
t1 = 60.0
dt = 0.05

tau = 2.5  # s
w = np.array([1.2, 0.4])  # m/s

t = np.arange(t0, t1 + dt, dt)


def simuluj_drift(r0, v0, t, dt, a_funkce):
    """
    Simulace driftu: Euler pro rychlost a lichoběžník pro polohu.
    """
    t = np.asarray(t, dtype=float)
    n = t.size
    r = np.zeros((n, 2), dtype=float)
    v = np.zeros((n, 2), dtype=float)
    a = np.zeros((n, 2), dtype=float)

    r[0] = np.asarray(r0, dtype=float)
    v[0] = np.asarray(v0, dtype=float)
    a[0] = a_funkce(v[0], r[0], t[0])

    for i in range(1, n):
        v[i] = v[i - 1] + a[i - 1] * dt
        r[i] = r[i - 1] + 0.5 * (v[i - 1] + v[i]) * dt
        a[i] = a_funkce(v[i], r[i], t[i])

    return r, v, a


def w_ext_const(r, t):
    """
    Konstantní rychlost prostředí $w$.
    """
    return w


def a_drift(v, r, t):
    """
    Zrychlení pro drift s odporem vůči prostředí.
    """
    a = np.zeros(2, dtype=float)
    ###### doplňte kód zde ######
    a = (1.0 / tau) * (w_ext_const(r, t) - v)
    #############################
    return a


r_drift, v_drift, a_drift_hist = simuluj_drift(r0, v0, t, dt, a_drift)

vykresli_drift(t, r_drift, v_drift, title="Úkol 3: drift s odporem vůči prostředí")


Pozorování: Trajektorie se nejprve rychle rozbíhá, ale velikost rychlosti se postupně stabilizuje. Pro konstantní $\mathbf{w}$ platí, že se $\mathbf{v}(t)$ blíží k $\mathbf{w}$, takže $|\mathbf{v}(t)| \to |\mathbf{w}|$. Z grafu velikosti rychlosti je vidět hladké přibližování bez oscilací.


---


## Rychlost prostředí jako časově proměnné vektorové pole

**Hlavní myšlenka:** Rychlost prostředí popíšeme vektorovým polem, které každému bodu a času přiřadí vektor rychlosti [m/s]:
$$
\mathbf{w}(\mathbf{r}, t) = (w_x(\mathbf{r}, t), w_y(\mathbf{r}, t)).
$$
Nehomogenní pole znamená, že se vektor liší v prostoru. Časově proměnné pole znamená, že se pole mění v čase (např. při průchodu fronty).

**Doplňující kontext:** Představte si oblast o velikosti zhruba 2000 × 2000 km rozdělenou na mřížku. V každém bodě známe vektor $\mathbf{w}$ a takový „snímek“ se opakuje třeba každý den po dobu 30 dní.

Následující dva satelitní snímky (měsíční průměry) ukazují skutečná data rychlosti větru nad oceánem; barvy reprezentují velikost a šipky směr vektorů rychlosti.

![Měsíční průměrné větry, listopad 2014](images/rapidscat_nov2014.jpg)
*Zdroj: NASA/JPL-Caltech, RapidScat (PIA20365), public domain.*

![Měsíční průměrné větry, listopad 2015](images/rapidscat_nov2015.jpg)
*Zdroj: NASA/JPL-Caltech, RapidScat (PIA20365), public domain.*


---


## Úkol 4: A co když se rychlost prostředí mění v prostoru?

**Hlavní myšlenka:** V reálné situaci se rychlost prostředí může měnit v závislosti na poloze (např. rychlostní pole kolem překážky). Zavedeme jednoduché pole $\mathbf{w}(\mathbf{r}, t)$, které částici „táhne“ směrem k bodu $\mathbf{r}^*$.

**Doplňující kontext:** Trajektorie pak bude zakřivená. Stejné numerické schéma funguje i pro složitější pole, jen je potřeba zvolit dostatečně malý krok $\Delta t$, aby trajektorie zůstala hladká.


In [ ]:
k = 1.5e-4  # 1/s
r_star = np.array([10_000.0, 5_000.0])


def w_ext_pole(r, t):
    """
    Příklad pole $w(\mathbf{r},t)$: rychlost prostředí směřuje k bodu r_star.
    """
    w_loc = np.zeros(2, dtype=float)
    ###### doplňte kód zde ######
    w_loc = k * (r_star - r)
    #############################
    return w_loc


def a_drift_pole(v, r, t):
    return (1.0 / tau) * (w_ext_pole(r, t) - v)


r_pole, v_pole, a_pole = simuluj_drift(r0, v0, t, dt, a_drift_pole)

vykresli_drift(t, r_pole, v_pole, title="Úkol 4: drift v poli rychlosti")


---


## Ukázka: časově proměnné vektorové pole rychlosti prostředí (syntetická data)

**Hlavní myšlenka:** Vytvoříme jednoduché pole rychlosti prostředí na mřížce a necháme se v něm unášet částici.

Budeme pracovat s oblastí 2000 × 2000 km a s časovou řadou 30 snímků (jeden snímek na den), kde je pole v každé buňce mřížky konstantní. Vektorové pole $\mathbf{w}(\mathbf{r}, t)$ chápeme jako rychlost prostředí [m/s].

Postup:
- vygenerujeme časovou řadu polí na mřížce,
- vizualizujeme několik časových snímků pomocí šipek,
- spočteme trajektorii v čase a vykreslíme ji.


### Pomocné funkce pro časově proměnné pole (`cv2_utils.py`)

**K čemu jsou:** Zabalí generování a vyhodnocení časově proměnného vektorového pole tak, aby v notebooku zůstala jen podstata úlohy.

**`generate_synthetic_w_grid(L, nx, ny, n_days, day)`**
- vytvoří syntetické pole na mřížce,
- vrací `x` a `y` [m], `t_grid` [s] a `W_grid` tvaru `(nt, ny, nx, 2)` v jednotkách [m/s].

**`w_from_grid(x, y, t, W_grid, x_grid, y_grid, t_grid)`**
- vrátí vektor rychlosti v bodě (x, y) a čase t,
- prostor i čas jsou po buňkách konstantní (nejbližší buňka na mřížce).

**`w_from_grid_mesh(X, Y, t, W_grid, x_grid, y_grid, t_grid, interpolate_time=True)`**
- vektorizované vyhodnocení na celé mřížce (užitečné pro vlastní quiver),
- v notebooku ho přímo nepoužíváme, ale je připravené v `cv2_utils.py`.

**`save_vector_field_animation(W_grid, x, y, t_grid, ...)`**
- uloží animaci šipek do mp4,
- vrátí cestu k uloženému souboru (vyžaduje `ffmpeg`),
- pro plynulost používá lineární interpolaci v čase.


In [ ]:
from cv2_utils import (
    generate_synthetic_w_grid,
    w_from_grid,
    save_vector_field_animation,
)


In [ ]:
# Generování syntetického pole na mřížce (větší oblast, bez dlouhodobě preferovaného směru)
L = 2_000_000.0  # m (2000 km => -1000 .. 1000 km)
nx, ny = 21, 21
DAY = 24 * 3600.0
n_days = 30

x, y, t_grid, W_grid = generate_synthetic_w_grid(
    L=L,
    nx=nx,
    ny=ny,
    n_days=n_days,
    day=DAY,
)


In [ ]:
# Hourly animation (shorter arrows, saved to mp4 and embedded)
out_path = save_vector_field_animation(
    W_grid,
    x,
    y,
    t_grid,
    out_path="vector_field_hourly.mp4",
    day=DAY,
)

from IPython.display import Video

Video(out_path, embed=True)


In [ ]:
# Trajektorie v časově proměnném poli
def w_ext_grid(r, t):
    """
    Rychlost prostředí w z časově proměnného pole na mřížce (konstantní v buňce).
    """
    return w_from_grid(r[0], r[1], t, W_grid, x, y, t_grid)

# Parametry simulace (odděleno od předchozích úkolů)
r0_field = np.array([0.0, 0.0])  # start uprostřed oblasti
v0_field = np.array([0.0, 0.0])

tau_field = 250.0  # s

t0_field = 0.0
t1_field = 7.0 * DAY

dt_sim = 30.0  # 30 s

t_field = np.arange(t0_field, t1_field + dt_sim, dt_sim)


def a_drift_cas(v, r, t):
    a = np.zeros(2, dtype=float)
    ###### doplňte kód zde ######
    a = (1.0 / tau_field) * (w_ext_grid(r, t) - v)
    #############################
    return a


r_td, v_td, a_td = simuluj_drift(r0_field, v0_field, t_field, dt_sim, a_drift_cas)

vykresli_drift(t_field, r_td, v_td, title='Drift v časově proměnném poli')


---


## Vliv časové konstanty $\tau$ na trajektorii

**Hlavní myšlenka:** Porovnáme několik hodnot $\tau$ se stejným startem a stejnou délkou simulace. $\tau$ reprezentuje, jak rychle se částice přizpůsobuje lokální rychlosti prostředí.

**Jak to číst:** Menší $\tau$ znamená rychlejší přizpůsobení k $\mathbf{w}$ (silnější tlumení relativního pohybu), větší $\tau$ naopak pomalejší přizpůsobení.


In [ ]:
# Porovnání různých hodnot tau pro stejný start a stejný čas
parametry = [
    ("tau = 200 s", 200.0),
    ("tau = 2000 s", 2000.0),
    ("tau = 12000 s", 12000.0),
    ("tau = 120000 s", 120000.0),
]

fig, ax = plt.subplots(figsize=(6, 6))

for label, tau_cmp in parametry:
    def a_drift_cmp(v, r, t, tau=tau_cmp):
        return (1.0 / tau) * (w_ext_grid(r, t) - v)

    r_cmp, v_cmp, _ = simuluj_drift(r0_field, v0_field, t_field, dt_sim, a_drift_cmp)
    ax.plot(r_cmp[:, 0] / 1000.0, r_cmp[:, 1] / 1000.0, label=f"{label}")

ax.scatter(r0_field[0] / 1000.0, r0_field[1] / 1000.0, color='black', s=20, label='start')
ax.set_xlabel('x [km]')
ax.set_ylabel('y [km]')
ax.set_title('Vliv časové konstanty tau na trajektorii v časově proměnném poli')
ax.set_aspect('equal')
ax.grid(True, alpha=0.3)
ax.legend(loc='best', fontsize=8)
plt.tight_layout()
plt.show()


---


## Shrnutí

**Hlavní body:**
- Umím formálně popsat $\mathbf{r}(t)$, $\mathbf{v}(t)$ a $\mathbf{a}(t)$ v 2D.
- Chápu, že jediná síla je odpor závislý na relativní rychlosti $\mathbf{v} - \mathbf{w}$.
- Vím, proč kombinujeme Eulerův krok pro $\mathbf{v}$ a lichoběžník pro $\mathbf{r}$.
- Umím volit rozumný krok $\Delta t$ a vím, že příliš velký krok zhorší přesnost.

**Doplňující poznámky (pro jistotu):**
- Rozumím významu časové konstanty $\tau$ a pole $\mathbf{w}$ a jejich vlivu na přizpůsobení rychlosti.
- Typické chyby: špatné jednotky, zapomenutá prealokace, pole špatného tvaru $(n, 2)$.
